In [ ]:
import os
# pyopencl will use the GPU on my Macbook Pro
os.environ['PYOPENCL_CTX'] = '1:2'

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from refnx.reflect import SLD, Structure, ReflectModel, use_reflect_backend
from refnx.analysis import Parameter, Objective, CurveFitter
from refnx.dataset import ReflectDataset
from tof_simulator import ReflectSimulator
from parabolic_brush import ParabolicBrush

In [ ]:
si = SLD(2.07)
sio2 = SLD(3.47)
dtol = SLD(5.6)
polymer = SLD(1.4)

# adsorbed amount
gamma = Parameter(95., 'adsorbed amount')
# roughness between brush and preceding layer
lrough = Parameter(10., 'left roughness')

# same brush as that in Karim1994.
brush = ParabolicBrush(polymer, 0.175, gamma, 1.0, 150, lrough, microslab_max_thickness=1)

In [ ]:
z = np.linspace(0, 1000, 1001)
vfp = brush.volume_fraction(z)
plt.plot(z, vfp);

In [ ]:
s = si | sio2(15, 3) | brush | dtol(0, 3)
s.contract = 1.5
model = ReflectModel(s, bkg=5e-7, dq=4.97)

In [ ]:
a0 = ReflectSimulator(model, 0.8, rebin=2.5)
a1 = ReflectSimulator(model, 3.5, rebin=2.5)

In [ ]:
with use_reflect_backend('pyopencl'):
    a0.run(8000000)
    for i in range(700):
        a1.run(1000000)
        if not i % 50:
            print(i)

In [ ]:
d0 = a0.reflectivity
d1 = a1.reflectivity
d0 += d1

In [ ]:
# set resolution of model (for constant dq/q smearing)
model.dq.value = np.mean(d0.x_err / d0.x) * 100

In [ ]:
d0.save('karim_simulation.dat')
plt.plot(d0.x, d0.y)
plt.plot(d0.x, model(d0.x))
plt.yscale('log')
plt.xscale('log');
# plt.xlim(0.01, 0.02)
# plt.ylim(0.1, 1.1);

In [ ]:
plt.plot(d0.x, (d0.y - model(d0.x))/d0.y_err)
plt.xscale('log')

In [ ]:
d0 = ReflectDataset('karim_simulation.dat')

In [ ]:
import FreeformVFP

In [ ]:
# a left slab for an interior layer
# we use such a layer to make a roughness between it and
# the preceding layer
lslab = polymer(thick=20, rough=lrough)
lslab.vfsolv.value = 0.5

fvp = FreeformVFP.FreeformVFP(95, [0.5, 0.5, 0.5], [0.1, 0.01, 0.01, 0.01], polymer, left_slabs=[lslab])

In [ ]:
# create a modelling structure using our FreeformVFP
s1 = si | sio2(10, 2) | lslab | fvp | dtol(0, 2)

# use profile contraction for speedier calculation
s1.contract = 1.5

# and a model
model1 = ReflectModel(s1, scale=1, bkg=1e-6, dq_type='constant')
model1.dq = np.mean(d0.x_err / d0.x) * 100

objective = Objective(model1, d0)

In [ ]:
plt.plot(*fvp.profile(), label='starting point of FreeformVFP analysis')
plt.plot(z, vfp, label='Karim profile')
plt.legend()

In [ ]:
objective.plot()
plt.yscale('log')

In [ ]:
objective.logp()

In [ ]:
# set some fitting parameters and bounds
# sio2 layer
s1[1].thick.setp(vary=True, bounds=(5, 20))
s1[1].rough.setp(vary=True, bounds=(1, 6))

# the interior slab layer
lslab.thick.setp(vary=True, bounds=(15, 30))
lslab.rough.setp(vary=True, bounds=(1, 15))
lslab.vfsolv.setp(vary=True, bounds=(0, 1))

# set limits on knot spacing:
for p in fvp.dzf:
    p.setp(vary=True, bounds=(0.01, 0.3))

# set limits on vf:
for p in fvp.vff:
    p.setp(vary=True, bounds=(0.05, 0.8))

# set limits on adsorbed amount (use +/- ~5%)
fvp.adsorbed_amount.setp(vary=True, bounds=(92, 98))

# roughness of d2o/brush (probably won't be sensitive to this)
s1[-1].rough.setp(vary=True, bounds=(1, 5))

model1.scale.setp(vary=True, bounds=(0.95, 1.05))
model1.bkg.setp(vary=True, bounds=(1e-8, 1e-6))

In [ ]:
fitter = CurveFitter(objective)

In [ ]:
fitter.fit('differential_evolution', target='nlpost');

In [ ]:
objective.plot()
plt.yscale('log')
plt.xscale('log')

In [ ]:
model1.threads = 1
fitter.initialise(pos='jitter')
fitter.sample(1000);

In [ ]:
fitter.reset()
fitter.sample(5, nthin=500);

In [ ]:
plt.plot(*fvp.profile(), label='ending point of FreeformVFP analysis')
plt.plot(z, vfp, label='Karim profile')
plt.legend();

In [ ]:
plt.plot(d0.x, (d0.y - model(d0.x))/d0.y_err)
plt.xscale('log')